In [ ]:
import pylightcurve as plc
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import sys
from scipy import stats
%matplotlib notebook

In [ ]:
#jd,raw_flux, raw_flux_err = np.loadtxt('Photometry_results.csv', delimiter=',', skiprows=1, usecols = [1,2,3], unpack=True)
jd,raw_flux, raw_flux_err = np.loadtxt('All_photometry.csv', delimiter=',', skiprows=1, usecols = [0,1,2], unpack=True)
std_fluxes = np.loadtxt('All_photometry.csv', delimiter=',', skiprows=1, usecols = [3,5,7,9,11,13,15,17,19,21,23,25], unpack=True)
std_fluxes_err = np.loadtxt('All_photometry.csv', delimiter = ',', skiprows=1, usecols = [4,6,8,10,12,14,16,18,20,22,24,26], unpack=True)

std_flux = np.sum(std_fluxes,axis=0)
std_flux_err = np.sqrt(np.sum(std_fluxes_err**2, axis=0))#std_flux1_err**2+std_flux2_err**2+std_flux3_err**2+std_flux4_err**2+std_flux5_err**2)
flux = raw_flux/std_flux
flux_err = flux*np.sqrt((raw_flux_err/raw_flux)**2+(std_flux_err/std_flux)**2)

flux_unc_std = np.std(flux[-18:]) #Uncertainty empirically found from the std of photos post-transit end
star_flux = flux[-18:]
jd_star = jd[-18:]
star_flux_avg = np.average(star_flux)
star_flux_avg_err = np.std(star_flux)/np.sqrt(18) #Standard Error of the Mean 
normalised_flux = flux/star_flux_avg
normalised_err = normalised_flux*np.sqrt((flux_err/flux)**2 + (star_flux_avg_err/star_flux_avg)**2)
normalised_std = flux_unc_std/star_flux_avg
###


In [ ]:
plt.figure()
plt.plot(jd, raw_flux, '.')
plt.xlabel("Time (JD)")
plt.ylabel("Flux (ADUs)")
plt.tight_layout()
plt.savefig('raw_lightcurve.jpg')



plt.figure()
plt.errorbar(jd, normalised_flux, yerr=normalised_err, fmt='.', capsize=3)
plt.xlabel("Time (JD)")
plt.ylabel("Relative flux")
plt.tight_layout()
plt.savefig('detrended_lightcurve2.jpg')




In [ ]:
def binning(time_adj, flux, flux_err,  n, expo=45):
    if type(flux_err) == np.float64:
        flux_err = np.asarray([flux_err] * len(flux))
    start_binning = time.process_time()
    print('Number of points:', len(flux))
    number = time_adj.size
    #We want to have n observations. Hence, each bin will include a 1/n of all observations
    bin_width_float = number/n
    bin_width = int(bin_width_float)
    i=0
    bin_start_arr = np.array([])
    bin_mid_arr = np.array([])
    binned_flux = np.array([])
    time_width_arr = np.array([])
    binned_flux_err = np.array([])
    int_time = np.array([])
    while i<=(n-1):  
        if i == 0:
            start = 0
        else:
            start = i*bin_width 
        if i == n-1:
            end = time_adj.size
        else:
            end = (i+1)*bin_width
        #print('Bin from number {0} to number {1}'.format(start,end))
        time_group = time_adj[start:end]
        time_width = time_group.max()-time_group.min()
        time_width_arr = np.append(time_width_arr, time_width)
        flux_group = flux[start:end]
        flux_err_group = flux_err[start:end]
        time_avg = np.average(time_group)
        flux_avg= np.average(flux_group)
        term_flux_bin_err = np.sqrt(np.sum(flux_err_group**2)/len(flux_err_group))
        flux_bin_err= term_flux_bin_err/np.sqrt(len(flux_group))
        #flux_bin_err= np.median(flux_err)/np.sqrt(len(flux_group))
        bin_start = time_group[0]
        bin_end = time_group[-1] + expo/86400
        bin_start_arr = np.append(bin_start_arr, bin_start)
        bin_mid_arr = np.append(bin_mid_arr, (bin_end+bin_start)/2)
        binned_flux = np.append(binned_flux, flux_avg)
        binned_flux_err = np.append(binned_flux_err, flux_bin_err)
        int_time = np.append(int_time, expo*len(flux_group))
        if i ==0:
            processed_tot = time_group.size
        else:
            processed_tot = processed_tot + time_group.size
        i +=1 
    int_time_avg =  stats.mode(int_time)[0][0] 
    print('Integration times:', int_time)
    print('All {0} bins have been created with total integration time {1} s, each spanning on average {2} s.'\
          .format(i, int_time_avg, 86400*np.median(time_width_arr)))
    save_bins =np.array([bin_start_arr, bin_mid_arr, binned_flux, binned_flux_err, int_time])
    return save_bins

In [ ]:
bins_wasp43 = binning(jd, normalised_flux, normalised_err, 21)
plt.figure()
plt.errorbar(bins_wasp43[1], bins_wasp43[2], yerr=bins_wasp43[3], fmt='.', capsize=3)
plt.xlabel('Time (JD)')
plt.ylabel('Relative flux')
plt.tight_layout()
plt.savefig('binned.jpg')

In [ ]:
def simulate(binned_obs, mag =12.305):
    planet = plc.get_planet('wasp43b')
    start_time, mid_time, flux, flux_unc, int_time = binned_obs
    planet.add_observation(
        time = mid_time,                # the time vector of our observation
                                    # np.array of float values 

        time_format = 'JD_UTC',    # format in which our time vector is expressed
                                    # str, available formats are: JD_UTC, MJD_UTC, HJD_UTC, HJD_TDB, BJD_UTC, BJD_TDB 

        exp_time = stats.mode(int_time)[0][0],             # exposure time of our time vector
                                    # float, in seconds

        time_stamp = 'mid',         # exposure time stamp for our time vector (do the numbers refer to the exposure start, the mid-exposure, or the exposure end?)
                                    # str, available stamps are: start, mid, end 

        flux = flux,                # the flux vector of our observation
                                    # np.array of float values, 

        flux_unc = flux_unc,        # the flux-uncertainty vector of our observation
                                    # np.array of float values, 

        flux_format = 'flux',       # format in which our flux and flux-uncertainty vectors are expressed
                                    # str, available formats are: flux, mag

        filter_name = 'luminance'   # filter used for this observation 
                                    # str, available filters are: all the default filters and those added manually by us
    )
    foldername = 'WASP43-08-03-22_TRY2'
    planet.transit_fitting(foldername)
    t_uncertainty = np.average(np.abs(np.loadtxt('./{0}/results.txt'.format(foldername), skiprows=14, usecols = [2,3])))
    return  t_uncertainty